<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/useful/good_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install exchange_calendars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import numpy as np
import requests
import datetime
import exchange_calendars as ecals # 개장일만
from io import StringIO
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler

In [6]:
XKRX = ecals.get_calendar("XKRX") # 한국 코드

250일 등락률, 거래대금 90-99: 범인매매

60일 등락률 50-100, 거래대금 10-50 : 조용히 오르는 애들 (내꺼)



In [11]:
class StockList():
  def __init__(self, period=250, increased=[.9, .99], traded=[.9, .99], pre_period=0):
    self.period = period
    self.increased = increased
    self.traded = traded
    self.pre_period = pre_period # 시작기점

    self.price_dic = {}

    self.start, self.today = self.get_date()
    self.df = self.get_stock_df()
    self.filtered_df = self.get_filtered_df()
    self.result_df = self.get_result_df()

  def get_date(self):
    today = datetime.date.today().strftime('%Y%m%d')
    if self.pre_period != 0:
      today = (datetime.date.today() - datetime.timedelta(days=self.pre_period)).strftime('%Y%m%d')
    start = (datetime.date.today() - datetime.timedelta(days=self.period)).strftime('%Y%m%d')

    if XKRX.is_session(today) == False:
      today = XKRX.previous_open(today).strftime('%Y%m%d')
    if XKRX.is_session(start) == False:
      start = XKRX.next_open(start).strftime('%Y%m%d')
    return start, today

  def get_stocks(self, market='STK'):
    data = {
      'mktId': market,
      'strtDd': self.start,
      'endDd': self.today,
      'money': '1',
      'adjStkPrc': '2',
      'adjStkPrc_check': 'Y',
      'share': '1',
      'csvxls_isNo': 'false',
      'name': 'fileDown',
      'url': 'dbms/MDC/STAT/standard/MDCSTAT01602'
    }
    gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    gen_key = requests.post(gen_url, data=data)

    down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
    r = requests.post(down_url, data={'code':gen_key.text})
    r.encoding = 'EUC-KR'
    return pd.read_csv(StringIO(r.text))

  def get_stock_df(self):
    return pd.concat([self.get_stocks(), self.get_stocks('KSQ')]).reset_index(drop=True)

  def get_filtered_df(self):
    traded_df = self.df[(self.df['거래대금'] < self.df['거래대금'].quantile(self.traded[1])) & (self.df['거래대금'] > self.df['거래대금'].quantile(self.traded[0]))]
    increased_df = self.df[(self.df['등락률'] > self.df['등락률'].quantile(self.increased[0])) & (self.df['등락률'] < self.df['등락률'].quantile(self.increased[1]))]
    selected = pd.Series(np.intersect1d(traded_df['종목명'].values, increased_df['종목명'].values))
    return self.df[self.df['종목명'].isin(selected)].sort_values('등락률', ascending=False).head(20)

  def get_sharpe(self, df):
    change = df['Change']+1
    return change.mean()/change.std()

  def get_sortino(self, df):
    change = df['Change']+1
    return change.mean()/(change[change<1]).std()

  def get_position(self, df):
    return df['Close'][-1]/df['Close'].max()

  def get_result_df(self):
    for i in self.filtered_df['종목코드']:
      self.price_dic[i] = fdr.DataReader(str(i), self.start, self.today)

    if (self.pre_period != 0):
      future_dic = {}
      for i in self.filtered_df['종목코드']:
        future = (datetime.date.today() + datetime.timedelta(days=self.pre_period)).strftime('%Y%m%d')
        future_dic[i] = fdr.DataReader(str(i), self.today, future)
      self.filtered_df['Performance'] = [(future_dic[x]['Close'][-1]/future_dic[x]['Close'][0]) for x in future_dic]
    else:
      self.filtered_df['Performance'] = [0 for x in self.price_dic]

    self.filtered_df['Sharpe'] = [self.get_sharpe(self.price_dic[x]) for x in self.price_dic]
    self.filtered_df['Sortino'] = [self.get_sortino(self.price_dic[x]) for x in self.price_dic]
    self.filtered_df['Position'] = [self.get_position(self.price_dic[x]) for x in self.price_dic]

    t = self.filtered_df.sort_values('Sharpe', ascending=False)

    scaler = MinMaxScaler(feature_range=(5, 10))

    t['Position'] = scaler.fit_transform(t[['Position']]).round(1)
    t['Sortino'] = scaler.fit_transform(t[['Sortino']]).round(1)
    t['Sharpe'] = scaler.fit_transform(t[['Sharpe']]).round(1)
    t['거래대금'] = scaler.fit_transform(t[['거래대금']]).round(1)
    t['등락률'] = scaler.fit_transform(t[['등락률']]).round(1)
    t['합산'] = t['Position'] + t['Sortino'] + t['Sharpe'] + t['거래대금'] + t['등락률']
    t['합산'] = scaler.fit_transform(t[['합산']]).round(1)

    t = t[['종목명', '종료일 종가', '등락률', '거래대금',	'Sharpe',	'Sortino',	'Position', '합산', 'Performance']].reset_index(drop=True).rename({'종료일 종가':'현재가', '등락률':'파워', '거래대금':'관심도', 'Sharpe':'Risk1', 'Sortino':'Risk2', 'Position':'모멘텀'}, axis=1)
    t.index+=1
    t.index.name='순위'

    return t

In [12]:
stocklist = StockList(250, [.99,1], [.99,1])
t = stocklist.result_df
t

,종목명,현재가,파워,관심도,Risk1,Risk2,모멘텀,합산,Performance
순위,,,,,,,,,
1,에코프로,623000,10.0,10.0,10.0,8.5,10.0,10.0,0
2,레인보우로보틱스,110500,5.0,5.0,8.6,10.0,8.7,7.0,0
3,금양,59700,8.4,6.2,5.0,5.0,5.0,5.0,0


In [ ]:
from google.colab import files

t.to_csv('king_df.csv', encoding="utf-8-sig") 
files.download('king_df.csv')

In [ ]:
stocklist = StockList()
t = stocklist.result_df
t

In [ ]:
t.to_csv('ai_df.csv', encoding="utf-8-sig") 
files.download('ai_df.csv')

In [ ]:
stocklist = StockList(period=250, increased=[.9, .99], traded=[.9, .99], pre_period=60)
t = stocklist.result_df
t = t[t['Performance'] > 1]
t

,종목명,현재가,파워,관심도,Risk1,Risk2,모멘텀,합산,Performance
순위,,,,,,,,,
1,메디톡스,215500,7.5,5.1,10.0,10.0,9.0,10.0,1.141531
2,삼아알미늄,39150,5.0,5.3,9.5,8.1,9.1,8.1,1.749681
3,에스피지,23800,5.0,6.1,9.3,8.5,10.0,8.9,1.338235
4,로보스타,32800,6.1,5.4,9.3,9.0,9.4,9.0,1.044207
5,바이오니아,46550,5.6,6.0,9.3,8.0,8.5,8.3,1.091300
6,에코플라스틱,4255,5.4,5.5,9.0,9.2,10.0,9.0,1.605170
7,AP위성,17800,5.5,5.4,8.5,9.4,8.2,8.1,1.028652
10,코스모화학,44200,7.3,8.3,8.3,7.2,8.6,9.2,1.251131
12,마인즈랩,23500,5.3,5.3,7.5,5.7,8.3,6.1,1.185106


In [ ]:
t.to_csv('profit60_df.csv', encoding="utf-8-sig") 
files.download('profit60_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
stocklist = StockList(period=250, increased=[.9, .99], traded=[.9, .99], pre_period=120)
t = stocklist.result_df
t = t[t['Performance'] > 1]
t

,종목명,현재가,파워,관심도,Risk1,Risk2,모멘텀,합산,Performance
순위,,,,,,,,,
1,오리온,122500,5.4,5.3,10.0,10.0,9.5,9.5,1.195102
3,현대두산인프라코어,7570,5.2,7.0,7.1,6.2,8.4,7.0,1.355350
4,삼아알미늄,40000,8.3,5.5,6.7,6.2,9.4,7.9,1.712500
5,현대건설기계,58600,7.6,5.5,6.7,6.3,8.8,7.4,1.119454
6,메리츠화재,46650,5.0,5.0,6.5,7.4,9.2,6.7,1.106109
7,태경비케이,5450,5.0,5.1,6.4,5.7,6.5,5.0,1.284404
8,삼천당제약,46850,6.4,5.5,6.3,7.1,9.4,7.3,1.376734
9,나무가,15050,6.5,5.3,6.1,5.8,7.7,6.0,1.017276
10,메리츠금융지주,38650,5.5,5.8,6.1,7.4,8.7,6.9,1.156533


In [ ]:
t.to_csv('profit120_df.csv', encoding="utf-8-sig") 
files.download('profit120_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>